In [7]:
%%capture
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')

In [8]:
%%stata -qui

use "../data/data", clear
rename log_flesch_kincaid_grade_level FKG
quietly tabulate year, generate(y_)
quietly tabulate cluster, generate(c_)

local journals  ecm jpe qje res  //AER based category

local jel_imp a_imp b_imp c_imp  e_imp f_imp g_imp h_imp i_imp j_imp k_imp /// 
		l_imp m_imp n_imp o_imp p_imp q_imp r_imp y_imp z_imp // D JEL based case

In [9]:
%%stata -qui
summarize flesch_kincaid_grade_level, detail
local cutoff = r(p90)
gen int FKG_01 = (flesch_kincaid_grade_level >= `cutoff')

In [11]:
%%stata -qui
#delimit ;
logit FKG_01 log_num_authors log_num_pages both_genders prop_women
			 `journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag, vce(cluster cluster);
#delimit cr

In [12]:
%%stata
estimates table, keep(log_num_authors log_num_pages both_genders prop_women) b(%5.4f) star varwidth(50) varlabel


-----------------------------------------------------------------
                                          Variable |   Active    
---------------------------------------------------+-------------
                Logarithm of the number of authors | -0.2194***  
                  Logarithm of the number of pages |  0.4182***  
    Indicator if author team includes both genders |  0.2597*    
             Proportion of women among the authors | -0.5166**   
-----------------------------------------------------------------
                         Legend: * p<0.05; ** p<0.01; *** p<0.001


In [13]:
%stata estat class


Logistic model for FKG_01

              -------- True --------
Classified |         D            ~D  |      Total
-----------+--------------------------+-----------
     +     |        25            16  |         41
     -     |       463          4045  |       4508
-----------+--------------------------+-----------
   Total   |       488          4061  |       4549

Classified + if predicted Pr(D) >= .5
True D defined as FKG_01 != 0
--------------------------------------------------
Sensitivity                     Pr( +| D)    5.12%
Specificity                     Pr( -|~D)   99.61%
Positive predictive value       Pr( D| +)   60.98%
Negative predictive value       Pr(~D| -)   89.73%
--------------------------------------------------
False + rate for true ~D        Pr( +|~D)    0.39%
False - rate for true D         Pr( -| D)   94.88%
False + rate for classified +   Pr(~D| +)   39.02%
False - rate for classified -   Pr( D| -)   10.27%
--------------------------------------------------